In [1]:
import os
import logging

import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as od
import numpy as np
import wandb
import optuna
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from tqdm import tqdm, trange

from datasets import CubeObstacle, CylinderObstacle, BlockageDataset
from utils.tools import calc_sig_strength_gpu, calc_loss
from utils.config import Hyperparameters as hp

/opt/anaconda3/envs/UAV_optim/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
random_seed = 42
torch.random.manual_seed(random_seed)
np.random.seed(random_seed)

batch_size = 2**10

In [3]:
class Net(nn.Module):
    def __init__(self, num_node, hidden_N, hidden_L, output_N=3):
        super(Net, self).__init__()
        self.hidden_N = hidden_N
        self.hidden_L = hidden_L
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(num_node, hidden_N))
        for _ in range(hidden_L):
            self.layers.append(nn.Linear(hidden_N, hidden_N))

        self.dropouts = nn.ModuleList()
        for _ in range(hidden_L):
            self.dropouts.append(nn.Dropout(0.3))

        self.batches = nn.ModuleList()
        for _ in range(hidden_L):
            self.batches.append(nn.BatchNorm1d(hidden_N))

        self.output = nn.Linear(hidden_N, output_N)
    def forward(self, x):
        z = x
        for layer, dropout, batch_norm in zip(self.layers, self.dropouts, self.batches):
            z = layer(z)
            z = batch_norm(z)
            z = F.leaky_relu(z, 0.05)
            z = dropout(z)

        z = torch.sigmoid(self.output(z))
        return z

In [4]:
obstacle_ls = [
    CubeObstacle(-30, 25, 35, 60, 20, 0.1),
    CubeObstacle(-30, -25, 45, 10, 35, 0.1),
    CubeObstacle(-30, -60, 35, 60, 20, 0.1),
    CubeObstacle(-70, -20, 20, 20, 65, 0.1),
    CubeObstacle(50, -20, 35, 25, 25, 0.1),
    CylinderObstacle(10, -5,  70, 15, 0.1),
    CylinderObstacle(-60, -50, 40, 20, 0.1),
    CylinderObstacle(60, -50, 30, 10, 0.1)
]

In [5]:
dataset = BlockageDataset(100000, obstacle_ls, 4, dtype=torch.float32).to(hp.device)
dataset.__len__()

100%|██████████| 100000/100000 [00:01<00:00, 59584.14it/s]


100000

In [6]:
data = dataset.gnd_nodes.cpu().numpy()
data = np.delete(data,2, axis=2).reshape(-1,8)
data

array([[ 16.193718 ,  45.977825 , -17.81342  , ...,  42.224365 ,
        -89.60678  ,  71.94956  ],
       [ -8.349337 , -47.516785 , -51.6471   , ..., -24.330484 ,
         49.94515  , -80.63625  ],
       [ -4.7886596,  98.33723  ,  54.08756  , ..., -53.275047 ,
        -18.087814 ,  56.016804 ],
       ...,
       [ 46.861378 ,  31.908358 ,  38.49393  , ..., -59.89102  ,
        -40.42146  , -36.707603 ],
       [ 48.58064  , -93.610695 ,  20.08375  , ...,  25.077305 ,
        -95.42112  ,  77.69216  ],
       [ -8.655481 ,   2.2979765, -89.57548  , ..., -79.84591  ,
        -44.92314  , -80.23318  ]], dtype=float32)

In [7]:
df = pd.DataFrame(data)
df

,0,1,2,3,4,5,6,7
0,16.193718,45.977825,-17.813419,25.843838,39.862583,42.224365,-89.606781,71.949562
1,-8.349337,-47.516785,-51.647099,-3.397177,77.366310,-24.330484,49.945148,-80.636253
2,-4.788660,98.337227,54.087559,-57.894302,0.288295,-53.275047,-18.087814,56.016804
3,54.244965,-54.379925,84.444664,18.513636,-24.015142,0.387116,21.014250,11.978106
4,-2.718993,0.986561,-64.717621,62.546581,8.352064,-73.784027,53.231419,86.488792
...,...,...,...,...,...,...,...,...
99995,-94.449883,-79.656647,27.009413,84.739967,-59.980907,-68.839775,27.179375,25.156099
99996,29.233435,-32.591549,72.588730,4.268974,42.914879,-63.389076,-90.642342,-11.339355
99997,46.861378,31.908358,38.493931,29.038160,6.288004,-59.891022,-40.421459,-36.707603
99998,48.580639,-93.610695,20.083750,-35.072330,37.721138,25.077305,-95.421120,77.692162


In [8]:
df.to_csv("./data/dataset.csv", index=False)

In [9]:
class TrainDataset(Dataset):
    def __init__(self, x, dtype=torch.float32):
        self.x = torch.tensor(x, dtype=dtype)

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx]

    def to(self, device: torch.device):
        self.x = self.x.to(device)
        return self

In [10]:
obst_points = []
for obstacle in obstacle_ls:
    obst_points.append(torch.tensor(obstacle.points, dtype=torch.float32))

obst_points = torch.cat([op for op in obst_points], dim=1).mT.to(hp.device)

In [26]:
torch.manual_seed(random_seed)
np.random.seed(random_seed)

x = pd.read_csv('./data/dataset.csv')
scaler_x = MinMaxScaler(feature_range=(0, 1))
x_scaled = scaler_x.fit_transform(x)

x_train, x_val = train_test_split(x_scaled, test_size=0.2, random_state=random_seed)

train_dataset = TrainDataset(x_train, dtype=torch.float32).to(hp.device)
val_dataset = TrainDataset(x_val, dtype=torch.float32).to(hp.device)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [31]:
import torch
import matplotlib.pyplot as plt
from tqdm import trange
import numpy as np

lr_ls = [1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
results = {lr: {"train_loss": [], "val_loss": []} for lr in lr_ls}

for lr in lr_ls:
    model = Net(train_dataset.x.shape[1], 1024, 4, output_N=2).to(hp.device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()  # 기본 손실 함수

    for epoch in trange(1000, desc=f"Training with lr={lr}"):
        train_loss = 0.0
        model.train()
        for x in train_dataloader:
            optimizer.zero_grad()
            y_pred = model(x)

            # x_reshaped 생성
            x_reshaped = torch.tensor(scaler_x.inverse_transform(x.cpu()), device=hp.device, dtype=torch.float32).view(-1, 4, 2)
            x_reshaped = torch.cat((x_reshaped, torch.zeros((x_reshaped.shape[0], x_reshaped.shape[1], 1), device=hp.device)), dim=-1)

            # y_pred 수정 및 손실 계산
            y_pred = torch.hstack((y_pred, torch.ones(y_pred.shape[0], 1, device=hp.device) * 0.7)) * 100
            loss = calc_loss(y_pred, x_reshaped, obst_points)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # 검증 손실 계산
        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for x in val_dataloader:
                y_pred = model(x)
                x_reshaped = torch.tensor(scaler_x.inverse_transform(x.cpu()), device=hp.device, dtype=torch.float32).view(-1, 4, 2)
                x_reshaped = torch.cat((x_reshaped, torch.zeros((x_reshaped.shape[0], x_reshaped.shape[1], 1), device=hp.device)), dim=-1)
                y_pred = torch.hstack((y_pred, torch.ones(y_pred.shape[0], 1, device=hp.device) * 0.7)) * 100
                val_loss += calc_loss(y_pred, x_reshaped, obst_points).item()

        # 에폭별 평균 손실 기록
        results[lr]["train_loss"].append(train_loss / len(train_dataloader))
        results[lr]["val_loss"].append(val_loss / len(val_dataloader))

# 시각화
plt.figure(figsize=(12, 6))
for lr in lr_ls:
    plt.plot(results[lr]["train_loss"], label=f"Train Loss (lr={lr})", linestyle="--")
    plt.plot(results[lr]["val_loss"], label=f"Val Loss (lr={lr})")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Train and Validation Loss for Different Learning Rates")
plt.legend()
plt.grid(True)
plt.show()


Training with lr=0.1:   0%|          | 1/1000 [01:41<28:10:18, 101.52s/it]


KeyboardInterrupt: 